In [2]:
import benepar 
parser = benepar.Parser("benepar_en2")
def parse_line(line):
    tree = parser.parse(line)
    item = None
    for tr in tree.subtrees():
        if tr.label() == 'VP':
            item = tr
    return item 


In [3]:
# Process queries by constituency parser, extract main verb (verb not in subordinating conjunction)
# Main objects are also extracted 
t = "person turns off the light as they're leaving"

from nltk.tree import ParentedTree
def is_sbar(tr, v_cnt):
    ptr = tr.parent()
    flag = False 
    while ptr:
        if ptr.label() == 'SBAR' and v_cnt !=0:
            flag = True 
            break 
        else:
            ptr = ptr.parent() 
    return flag 

verbs_tags = ['VBZ', 'VBP', 'VB', 'VBD','VBG', 'VBN']
nouns_tags = ['NNS', 'NN']
def find_verb(item):
    verb = None 
    for st in item.subtrees():
        if st.label() in verbs_tags:
            verb = st.leaves()
            return verb, st  
    return verb, st  

def find_noun(item):
    noun = None 
    for st in item.subtrees():
        if st.label() in nouns_tags:
            noun = st.leaves()
            return noun
    return None 

def process_line(t):
    line = t.split("##")[-1]
    tree = parser.parse(line)
    newtree = ParentedTree.convert(tree)
    output= [] 
    verbs = [] 
    nouns = [] 
    v_cnt = 0 
    for tr in newtree.subtrees():
        if tr.label() == 'VP':
            flag = is_sbar(tr, v_cnt)
            if not flag:
                verb, st = find_verb(tr)
                if verb not in verbs:
                    verbs.append(verb)
                    v_cnt +=1 
                noun = find_noun(tr)
                if noun not in nouns: 
                    nouns.append(noun)
                
    return verbs, nouns  


In [3]:
process_line(lines[590])

NameError: name 'lines' is not defined

In [4]:
test_verb = {}
test_nouns = {} 

test_lines = open("/Users/yanjungao/Desktop/VPMT/data/charades/annotations/charades_sta_test.txt").readlines()

for l in test_lines:
    verb, noun = process_line(l)
    test_verb[test_lines.index(l)] = verb 
    test_nouns[test_lines.index(l)] = noun

train_verb = {}
train_nouns = {} 

train_lines = open("/Users/yanjungao/Desktop/VPMT/data/charades/annotations/charades_sta_train.txt").readlines()
for l in train_lines:
    verb, noun = process_line(l)
    train_verb[train_lines.index(l)] = verb 
    train_nouns[train_lines.index(l)] = noun

In [5]:
queris_verb

NameError: name 'queris_verb' is not defined

In [ ]:
lines[66]

In [ ]:
tree = parser.parse(lines[66].split("##")[-1])
print(tree)

In [ ]:
verb, noun = process_line(lines[662])
print(verb)

In [ ]:
from nltk.stem import SnowballStemmer 
stemmer = SnowballStemmer('english')

In [8]:
stem_queries_verb = {}
from nltk.stem.wordnet import WordNetLemmatizer
linking_verb = ['is', 'was', 'be', 'are', 'seem', 'start','begin','have']
def orgainze(queris_verb, alls):
    """ Remove none, linking verbs"""
    stem_queries_verb = {}
    for k,v in queris_verb.items():
        stem_queries_verb[k] = []
        verb_set = [] 
        for verb in v:
            if verb:
                stemmed = WordNetLemmatizer().lemmatize(verb[0],'v')
                if stemmed not in linking_verb:
                    stem_queries_verb[k].append(stemmed)
                    verb_set.append(stemmed)
                else:
                    stem_queries_verb[k].append(None)
            else:
                stem_queries_verb[k].append(None)
                verb_set=[None]
        alls.append(verb_set)
    return stem_queries_verb, alls 
        

In [ ]:
WordNetLemmatizer().lemmatize('kept','v')

In [9]:
alls = [] 
train_stem_verbs, alls = orgainze(train_verb, alls)
test_stem_verbs, alls = orgainze(test_verb, alls)

In [ ]:
train_stem_verbs

In [10]:
obj_alls = [] 
train_stem_objs, obj_alls = orgainze(train_nouns, obj_alls)
test_stem_objs, obj_alls = orgainze(test_nouns, obj_alls)


In [11]:
neat_alls = []

def get_vocab(alls):
    vocab_alls = {} 
    for i in alls:
        if len(i) == 1:
            item = i[0]
            if item in vocab_alls:
                vocab_alls[item] +=1 
            else:
                vocab_alls[item] = 1 
        else:
            print(i)
            try:
                item="_".join(i)
            except:
                i.remove(None)
                item="_".join(i)
            if item in vocab_alls:
                vocab_alls[item] +=1 
            else:
                vocab_alls[item] = 1 
    return vocab_alls 

def get_vocab_single(alls):
    vocab_alls = {} 
    for i in alls:
        if len(i) == 1:
            item = i[0]
            if item in vocab_alls:
                vocab_alls[item] +=1 
            else:
                vocab_alls[item] = 1 
        else:
            print(i)
            try:
                i.remove(None)
            except:
                pass 
            #item="_".join(i)
            for item in i:
                if item in vocab_alls:
                    vocab_alls[item] +=1 
                else:
                    vocab_alls[item] = 1 
    return vocab_alls


vocab_nouns = get_vocab(obj_alls)
vocab_verbs = get_vocab(alls)

['couch', 'television']
['sofa', 'dish']
['door', 'doorknob']
['chair', 'laptop']
['chair', 'pillow']
['room', 'box']
['foot', 'door']
[]
['laptop', 'sandwich']
['chair', 'light']
['doorway', 'towel']
['bag', 'shoe']
['door', 'bag']
['book', 'turn']
['pillow', 'floor']
['toilet', 'book']
['stave', 'food']
['chair', 'dish']
['doorway', 'towel']
['table', 'sandwich']
['table', 'sandwich']
['food', 'shelf']
[]
['bed', 'food']
['sofa', 'pillow']
['table', 'something']
['laptop', 'stave']
['laptop', 'refrigerator']
['window', 'sofa']
['room', 'pillow']
['person', 'snack']
['door', 'towel']
['din', 'book']
['door', 'doorknob']
['jacket', 'blanket']
[]
['doorway', 'sandwich']
['kitchen', 'food']
['pantry', 'sandwich']
[None, 'glass']
['encounter', 'phone']
['hallway', 'bag']
['room', 'laptop']
['door', 'grocery']
['chair', 'boot']
['floor', 'pillow']
['kitchen', 'something']
['bathroom', 'hand']
['hand', 'mirror']
['sofa', 'phone']
['shoe', 'door']
['sofa', 'book']
['chair', 'food']
['bathroo

['sit', 'eat']
['stand', 'eat']
['grab', 'open']
['sit', 'undress']
['open', 'hold']
['stand', 'hold']
['stand', 'fix']
['sit', 'eat']
['open', 'get']
['stop', 'look']
['sit', 'eat']
['sit', 'hold']
['walk', 'hold']
['pick', 'talk']
['walk', 'drink']
['take', 'make']
['play', 'take']
['sit', 'eat']
['stand', 'drink']
['sit', 'eat']
['talk', 'put']
['stand', 'laugh']
['talk', 'use']
['walk', 'hold']
['try', 'fix']
['turn', 'turn']
['stand', 'drink']
['open', 'contain']
['run', 'eat']
['put', 'take']
['work', 'read']
['sit', 'drink']
['put', 'close']
['close', 'walk']
['try', 'cook']
['hold', 'twirl']
['run', 'hold']
['laugh', 'hold']
['check', 'smile']
['run', 'grab']
['see', 'undress']
['walk', 'hold']
['walk', 'hold']
['run', 'grab']
['run', 'look']
['decide', 'grasp']
['grab', 'take']
['sit', 'eat']
['run', 'sit']
['stop', 'watch']
['dress', 'put']
['stand', 'undress']
['sit', 'put']
['react', 'laugh']
['open', 'get']
['decide', 'put']
['open', 'grab']
['walk', 'look']
['sit', 'eat']

In [12]:
vocab_nouns

{'book': 564,
 'phone': 327,
 'glass': 520,
 'water': 101,
 'television': 112,
 'cup': 297,
 'mirror': 81,
 None: 1731,
 'camera': 127,
 'notebook': 6,
 'door': 1179,
 'shoe': 485,
 'stave': 98,
 'something': 208,
 'coffee': 80,
 'room': 135,
 'couch_television': 5,
 'bed': 148,
 'laptop': 411,
 'table': 153,
 'sofa_dish': 1,
 'door_doorknob': 3,
 'light': 769,
 'medicine': 142,
 'chair': 350,
 'refrigerator': 229,
 'chair_laptop': 4,
 'clothe': 364,
 'sandwich': 425,
 'blanket': 260,
 'food': 436,
 'blanket/pillow': 1,
 'chair_pillow': 4,
 'pillow': 335,
 'bag': 382,
 'floor': 155,
 'room_box': 5,
 'box': 364,
 'cabinet': 372,
 'foot_door': 1,
 '': 39,
 'laptop_sandwich': 1,
 'shelf': 95,
 'closet': 195,
 'doorway': 258,
 'bowl': 13,
 'detergent': 3,
 'load': 2,
 'washer': 3,
 'seat': 4,
 'basement': 9,
 'front': 51,
 'chair_light': 1,
 'window': 407,
 'card': 2,
 'groceries': 66,
 'pan': 3,
 'pair': 55,
 'wardrobe': 14,
 'bathroom': 54,
 'can': 4,
 'bedroom': 43,
 'doorway_towel': 4,

In [13]:
alls = [] 
train_stem_verbs, alls = orgainze(train_verb, alls)
test_stem_verbs, alls = orgainze(test_verb, alls)
obj_alls = [] 
train_stem_objs, obj_alls = orgainze(train_nouns, obj_alls)
test_stem_objs, obj_alls = orgainze(test_nouns, obj_alls)

vocab_nouns = get_vocab_single(obj_alls)
vocab_verbs = get_vocab_single(alls)


['couch', 'television']
['sofa', 'dish']
['door', 'doorknob']
['chair', 'laptop']
['chair', 'pillow']
['room', 'box']
['foot', 'door']
[]
['laptop', 'sandwich']
['chair', 'light']
['doorway', 'towel']
['bag', 'shoe']
['door', 'bag']
['book', 'turn']
['pillow', 'floor']
['toilet', 'book']
['stave', 'food']
['chair', 'dish']
['doorway', 'towel']
['table', 'sandwich']
['table', 'sandwich']
['food', 'shelf']
[]
['bed', 'food']
['sofa', 'pillow']
['table', 'something']
['laptop', 'stave']
['laptop', 'refrigerator']
['window', 'sofa']
['room', 'pillow']
['person', 'snack']
['door', 'towel']
['din', 'book']
['door', 'doorknob']
['jacket', 'blanket']
[]
['doorway', 'sandwich']
['kitchen', 'food']
['pantry', 'sandwich']
[None, 'glass']
['encounter', 'phone']
['hallway', 'bag']
['room', 'laptop']
['door', 'grocery']
['chair', 'boot']
['floor', 'pillow']
['kitchen', 'something']
['bathroom', 'hand']
['hand', 'mirror']
['sofa', 'phone']
['shoe', 'door']
['sofa', 'book']
['chair', 'food']
['bathroo

['undress', 'take']
['undress', 'take']
['go', 'sink', 'wash']
['walk', 'wrap']
['sneeze', 'look']
['run', 'open']
['eat', 'cuddle']
['sit', 'eat']
['sit', 'eat']
['sit', 'watch']
['run', 'hold']
['try', 'turn']
[]
['eat', 'turn']
['walk', 'eat']
['take', 'follow']
['open', 'look']
['undress', 'remove']
['stand', 'dress']
['walk', 'drink']
['work', 'sneeze']
['see', 'sneeze']
['finish', 'dress']
['proceed', 'take']
['walk', 'open']
['see', 'watch']
['get', 'open', 'get']
['hold', 'take']
['lie', 'awaken']
['take', 'direct']
['sit', 'hold']
['sit', 'tap']
['sit', 'watch']
['go', 'eat']
['use', 'tidy']
['look', 'adjust']
['throw', 'shake']
['sit', 'eat']
['walk', 'hold']
['proceed', 'close']
['finish', 'eat']
['open', 'find']
['read', 'close']
['undress', 'take']
['lie', 'awaken']
['continue', 'turn']
['enter', 'hold']
['awaken', 'hold']
['wake', 'sneeze']
['get', 'dress']
['put', 'sneeze']
['sit', 'drink']
['get', 'dress']
['sit', 'read']
['struggle', 'get']
['struggle', 'open']
['see',

In [14]:
total = len(alls)
percentage_verbs = {k:v/total for k,v in vocab_verbs.items()}
percentage_verbs['put'] * total
len(vocab_verbs)

196

In [15]:
len(vocab_verbs)
vocab_verb_idx = {}
cnt= 0
for k,v in vocab_verbs.items():
    vocab_verb_idx[cnt] = k 
    cnt += 1 

vocab_obj_idx = {}
cnt= 0
for k,v in vocab_nouns.items():
    vocab_obj_idx[cnt] = k 
    cnt += 1 
    

In [20]:
len(vocab_nouns)

371

In [16]:
"""
Identifying threshold 
"""

import numpy as np 
thre = np.mean(list(vocab_verbs.values()))
low_thres = [] 
for k,v in vocab_verbs.items():
        if v < thre:
            low_thres.append(k)
            #print(k)
            
high_thres = [] 
for k,v in vocab_verbs.items():
        if v > thre:
            high_thres.append(k)
            
print(len(high_thres))
print(len(low_thres))

30
166


In [17]:
from nltk.corpus import wordnet

syns = wordnet.synsets("continue")
synonyms = {}
for item in high_thres:
    if item:
        synonyms[item] = [] 
        for syn in wordnet.synsets(item):
            for l in syn.lemmas():
                if l.name() in low_thres and l.name() not in synonyms[item]:
                    synonyms[item].append(l.name())
                    
low_synonyms = {} 
for item in low_thres:
    if item:
        low_synonyms[item] = [] 
        for syn in wordnet.synsets(item):
            for l in syn.lemmas():
                if l.name() in low_thres and l.name() not in low_synonyms[item]:
                    low_synonyms[item].append(l.name())
                    

In [18]:
synonyms

{'put': ['set', 'place', 'pose', 'lay'],
 'play': ['bring', 'work'],
 'pour': [],
 'watch': ['follow', 'see', 'check'],
 'take': ['return',
  'lead',
  'direct',
  'bring',
  'remove',
  'consume',
  'make',
  'carry',
  'pack',
  'contain'],
 'sneeze': [],
 'awaken': ['wake', 'awake'],
 'hold': ['grasp',
  'wait',
  'keep',
  'make',
  'give',
  'carry',
  'contain',
  'check'],
 'sit': ['pose', 'seat'],
 'open': ['give'],
 'tidy': ['straighten'],
 'smile': [],
 'cook': ['make', 'prepare'],
 'run': ['go', 'lead', 'work', 'lean', 'carry', 'move'],
 'drink': ['swallow'],
 'eat': ['consume'],
 'turn': ['bend', 'go', 'work'],
 'close': ['finish', 'end', 'shut', 'fold'],
 'undress': ['peel'],
 'snuggle': ['cuddle'],
 'walk': [],
 'stand': [],
 'throw': ['drop', 'give', 'flip', 'switch', 'make'],
 'laugh': [],
 'read': [],
 'wash': [],
 'get': ['go', 'let', 'find', 'come', 'bring', 'make', 'cause', 'pose'],
 'dress': ['set', 'do'],
 'look': ['appear', 'see', 'wait'],
 'fix': ['set', 'make',

In [19]:
low_synonyms

{'see': ['see', 'find', 'check', 'examine'],
 'grasp': ['grasp', 'reach'],
 'appear': ['appear'],
 'swallow': ['swallow'],
 'work': ['work', 'go', 'make', 'bring', 'form'],
 "'re": [],
 'stop': ['stop', 'check', 'point', 'contain', 'end', 'finish'],
 'use': ['use'],
 'lay': ['lay', 'set', 'place', 'pose', 'lie'],
 'debate': ['debate'],
 'lie': ['lie'],
 'keep': ['keep', 'continue', 'proceed'],
 'place': ['place', 'seat', 'set', 'pose', 'lay', 'direct', 'point'],
 'enter': ['enter'],
 'talk': ['talk'],
 'flip': ['flip', 'toss', 'switch', 'flick'],
 'leave': ['leave', 'lead', 'give'],
 'continue': ['continue', 'proceed', 'keep', 'remain', 'cover'],
 'fold': ['fold', 'bend'],
 'grab': ['grab'],
 'carry': ['carry', 'pack', 'contain'],
 'write': ['write'],
 'try': ['attempt', 'try', 'examine'],
 'approach': ['approach'],
 'redress': ['redress'],
 'pick': ['pick', 'clean'],
 'straighten': ['straighten'],
 'decide': ['decide'],
 'move': ['move', 'go', 'proceed'],
 'stretch': ['stretch', 'reac

In [20]:
def make_vocab_index(vocab_alls):
    cnt = 0
    action_index = {}
    for k in list(vocab_alls.keys()):
        action_index[cnt] = k 
        cnt +=1 
    action_index[cnt] = None 
    #action_index[cnt+1] = 'wear'
    action_v_ind = {v:k for k,v in action_index.items()}
    
    return action_index , action_v_ind

obj_index, obj_v_ind = make_vocab_index(vocab_nouns)
action_index, action_v_ind = make_vocab_index(vocab_verbs)

In [67]:
vocab_nouns

{'book': 621,
 'phone': 364,
 'glass': 545,
 'water': 109,
 'television': 139,
 'cup': 311,
 'mirror': 94,
 None: 1731,
 'camera': 143,
 'notebook': 7,
 'door': 1253,
 'shoe': 510,
 'stave': 111,
 'something': 236,
 'coffee': 91,
 'room': 222,
 'couch': 132,
 'bed': 191,
 'laptop': 446,
 'table': 192,
 'sofa': 122,
 'dish': 163,
 'doorknob': 53,
 'light': 786,
 'medicine': 151,
 'chair': 429,
 'refrigerator': 249,
 'clothe': 383,
 'sandwich': 512,
 'blanket': 287,
 'food': 488,
 'blanket/pillow': 1,
 'pillow': 384,
 'bag': 433,
 'floor': 183,
 'box': 400,
 'cabinet': 392,
 'foot': 4,
 'shelf': 98,
 'closet': 221,
 'doorway': 298,
 'bowl': 14,
 'detergent': 5,
 'load': 2,
 'washer': 3,
 'seat': 4,
 'basement': 18,
 'front': 52,
 'window': 424,
 'card': 2,
 'groceries': 71,
 'pan': 3,
 'pair': 55,
 'wardrobe': 16,
 'bathroom': 81,
 'can': 4,
 'bedroom': 65,
 'towel': 251,
 'turn': 4,
 'person': 49,
 'snack': 42,
 'drink': 156,
 'cleaner': 1,
 'stuff': 6,
 'broom': 179,
 'hair': 70,
 'din

In [59]:
tmp = {k: v for k, v in sorted(vocab_nouns.items(), key=lambda item: item[1])}
len(tmp)

371

In [1]:
tmp = {k: v for k, v in sorted(vocab_verbs.items(), key=lambda item: item[1])}
len(tmp)

NameError: name 'vocab_verbs' is not defined

In [60]:
def mark_vocab_index(test_stem_verbs,action_v_ind):
    test_verb_ids = {}
    for k,v in test_stem_verbs.items():
        if len(v) == 0:
            test_verb_ids[k] = action_v_ind[None] 
        elif len(v) == 1:
            if v[0] not in action_v_ind:
                action_v_ind[v[0]] = len(action_v_ind) + 1
            else:
                test_verb_ids[k] = action_v_ind[v[0]]
        else:
            if None in v:
                v.remove(None)
            print("clean v", v)
            if len(v) == 1:
                if v[0] not in action_v_ind:
                    action_v_ind[v[0]] = len(action_v_ind) + 1
                else:
                    test_verb_ids[k] = action_v_ind[v[0]]
            else:
                item="_".join(v)
                if item not in action_v_ind:
                    action_v_ind[item] = len(action_v_ind) + 1
                else:
                    test_verb_ids[k] = action_v_ind[item]
    return test_verb_ids 
                
test_obj_ids = mark_vocab_index(test_stem_objs, obj_v_ind)
train_obj_ids = mark_vocab_index(train_stem_objs, obj_v_ind)

clean v ['window']
clean v ['bathroom', 'glass']
clean v ['door']
clean v ['doorway', 'coffee']
clean v ['entryway', 'sandwich']
clean v ['doorway', 'glass']
clean v ['house', 'food']
clean v ['dress']
clean v ['top', 'laptop']
clean v ['stairs', 'bag']
clean v ['bed']
clean v ['bed']
clean v ['phone', 'talk']
clean v ['house', 'bag']
clean v ['room', 'sandwich']
clean v ['bathroom', 'dish']
clean v ['box']
clean v ['book']
clean v ['blanket']
clean v ['doorknob', 'cabinet']
clean v ['door', 'snack']
clean v ['oven']
clean v ['basement']
clean v ['floor']
clean v ['television', 'apple']
clean v ['toilet']
clean v ['bathroom', 'cup']
clean v ['garage', 'pillow']
clean v ['shoe']
clean v ['room']
clean v ['towel', 'counter']
clean v ['bathroom', 'phone']
clean v ['entryway', 'glass']
clean v ['floor', 'phone']
clean v ['floor', 'book']
clean v ['room', 'food']
clean v ['chair', 'refrigerator']
clean v ['bed', 'person']
clean v ['chair', 'something']
clean v ['kitchen']
clean v ['doorway'

clean v ['door']
clean v ['chair']
clean v ['kitchen', 'refrigerator']
clean v ['couch', 'television']
clean v ['picture']
clean v ['hallway']
clean v ['water', 'drink']
clean v ['sandwich', 'door']
clean v ['stave']
clean v ['refrigerator', 'door']
clean v ['hallway', 'blanket']
clean v ['blanket', 'room']
clean v ['food', 'camera']
clean v ['bedroom']
clean v ['window']
clean v ['pillow', 'doorway']
clean v ['bed', 'pillow']
clean v ['bed', 'blanket']
clean v ['window']
clean v ['towel', 'water']
clean v ['stairs', 'broom']
clean v ['sandwich']
clean v ['coffee', 'drink']
clean v ['door']
clean v ['chair', 'box']
clean v ['person', 'pillow']
clean v ['sofa']
clean v ['light', 'napkin']
clean v ['carpet', 'sandwich']
clean v ['door']
clean v ['house', 'laptop']
clean v ['door', 'laptop']
clean v ['laundry', 'picture']
clean v ['room', 'picture']
clean v ['basement', 'bag']
clean v ['stairs', 'sandwich']
clean v ['camera', 'picture']
clean v ['camera', 'picture']
clean v ['garage', 'co

In [61]:
train_obj_ids

{0: 0,
 1: 1,
 2: 1,
 3: 2,
 4: 3,
 5: 3,
 6: 4,
 7: 5,
 8: 6,
 9: 371,
 10: 6,
 11: 8,
 12: 47,
 13: 371,
 14: 9,
 15: 371,
 16: 10,
 17: 11,
 18: 11,
 19: 1,
 20: 11,
 21: 53,
 22: 12,
 23: 13,
 24: 14,
 25: 15,
 26: 371,
 27: 11,
 28: 10,
 29: 4,
 30: 4,
 31: 480,
 32: 10,
 33: 17,
 34: 18,
 35: 18,
 36: 18,
 37: 2,
 38: 0,
 39: 19,
 40: 3,
 42: 10,
 43: 11,
 44: 11,
 45: 10,
 46: 11,
 48: 23,
 49: 2,
 50: 2,
 51: 24,
 52: 24,
 53: 10,
 54: 10,
 55: 25,
 56: 2,
 57: 26,
 58: 4,
 59: 371,
 60: 18,
 61: 18,
 63: 27,
 64: 27,
 65: 11,
 66: 13,
 67: 19,
 68: 5,
 69: 2,
 70: 28,
 71: 29,
 72: 26,
 73: 26,
 74: 26,
 75: 371,
 76: 30,
 77: 27,
 78: 17,
 79: 31,
 80: 371,
 81: 25,
 82: 25,
 84: 32,
 85: 33,
 86: 33,
 87: 33,
 88: 33,
 89: 34,
 90: 33,
 92: 35,
 93: 29,
 94: 35,
 95: 36,
 96: 371,
 97: 36,
 98: 35,
 99: 32,
 100: 28,
 101: 28,
 103: 10,
 104: 27,
 105: 8,
 106: 35,
 107: 14,
 108: 371,
 109: 10,
 110: 371,
 112: 0,
 113: 25,
 114: 27,
 115: 25,
 116: 27,
 117: 38,
 118: 8,
 

In [68]:
def mark_vocab_index_multiple(test_stem_verbs,action_v_ind):
    test_verb_ids = {}
    for k,v in test_stem_verbs.items():
        if len(v) == 0:
            test_verb_ids[k] = action_v_ind[None] 
        elif len(v) == 1:
            if v[0] not in action_v_ind:
                action_v_ind[v[0]] = len(action_v_ind) + 1
            else:
                test_verb_ids[k] = action_v_ind[v[0]]
        else:
            if None in v:
                v.remove(None)
            #print("clean v", v)
            if len(v) == 1:
                if v[0] not in action_v_ind:
                    action_v_ind[v[0]] = len(action_v_ind) + 1
                else:
                    test_verb_ids[k] = action_v_ind[v[0]]
            else:
                #item="_".join(v)
                test_verb_ids[k] = [] 
                for item in v:
                    if item not in action_v_ind:
                        action_v_ind[item] = len(action_v_ind) + 1
                    else:
                        test_verb_ids[k].append(action_v_ind[item]) 
    return test_verb_ids 

test_obj_ids = mark_vocab_index_multiple(test_stem_objs, obj_v_ind)
train_obj_ids = mark_vocab_index_multiple(train_stem_objs, obj_v_ind)


In [70]:
test_stem_objs

{0: ['light'],
 1: ['light'],
 2: ['light'],
 3: ['switch'],
 4: ['picture'],
 5: ['bag'],
 6: ['doorway'],
 7: ['door'],
 8: ['door'],
 9: ['door'],
 10: ['stairs'],
 11: ['stairs'],
 12: ['window'],
 13: ['chair'],
 14: ['window'],
 15: ['door'],
 16: ['chair'],
 17: ['chair'],
 18: ['bathroom', 'glass'],
 19: ['bag'],
 20: ['chair'],
 21: ['chair'],
 22: ['blanket'],
 23: ['clothe'],
 24: [None],
 25: ['sofa'],
 26: ['medicine'],
 27: ['chair'],
 28: ['cup'],
 29: ['door'],
 30: ['door'],
 31: ['cup'],
 32: ['doorway', 'coffee'],
 33: ['glass'],
 34: ['shoe'],
 35: ['glass'],
 36: ['food'],
 37: ['entryway', 'sandwich'],
 38: ['glass'],
 39: ['cup'],
 40: ['doorway', 'glass'],
 41: ['sandwich'],
 42: ['shoe'],
 43: ['cup'],
 44: ['house', 'food'],
 45: ['dress'],
 46: ['glass'],
 47: ['drink'],
 48: ['bed'],
 49: [None],
 50: ['bed'],
 51: ['pair'],
 52: ['shoe'],
 53: ['book'],
 54: ['top', 'laptop'],
 55: ['stairs', 'bag'],
 56: ['door'],
 57: ['doorway'],
 58: ['din'],
 59: ['bag

In [64]:
obj_v_ind

{'book': 0,
 'phone': 1,
 'glass': 2,
 'water': 3,
 'television': 4,
 'cup': 5,
 'mirror': 6,
 None: 371,
 'camera': 8,
 'notebook': 9,
 'door': 10,
 'shoe': 11,
 'stave': 12,
 'something': 13,
 'coffee': 14,
 'room': 15,
 'couch': 16,
 'bed': 17,
 'laptop': 18,
 'table': 19,
 'sofa': 20,
 'dish': 21,
 'doorknob': 22,
 'light': 23,
 'medicine': 24,
 'chair': 25,
 'refrigerator': 26,
 'clothe': 27,
 'sandwich': 28,
 'blanket': 29,
 'food': 30,
 'blanket/pillow': 31,
 'pillow': 32,
 'bag': 33,
 'floor': 34,
 'box': 35,
 'cabinet': 36,
 'foot': 37,
 'shelf': 38,
 'closet': 39,
 'doorway': 40,
 'bowl': 41,
 'detergent': 42,
 'load': 43,
 'washer': 44,
 'seat': 45,
 'basement': 46,
 'front': 47,
 'window': 48,
 'card': 49,
 'groceries': 50,
 'pan': 51,
 'pair': 52,
 'wardrobe': 53,
 'bathroom': 54,
 'can': 55,
 'bedroom': 56,
 'towel': 57,
 'turn': 58,
 'person': 59,
 'snack': 60,
 'drink': 61,
 'cleaner': 62,
 'stuff': 63,
 'broom': 64,
 'hair': 65,
 'din': 66,
 'pot': 67,
 'car': 68,
 'va

In [25]:
test_stem_verbs
"""getting dict[query id]: verb and its synonyms   """
def mark_vocab_syns(test_stem_verbs,low_synonyms, high_synonyms):
    test_verb_ids = {}
    for k,v in test_stem_verbs.items():
        if len(v) == 0:
            test_verb_ids[k] = None 
        elif len(v) == 1:
            if v[0] in high_synonyms:
                if len(high_synonyms[v[0]]) == 0:
                    test_verb_ids[k] = v 
                else:
                    test_verb_ids[k] = high_synonyms[v[0]]
            elif v[0] in low_synonyms:
                if len(low_synonyms[v[0]]) == 0:
                    test_verb_ids[k] = v 
                else:
                    test_verb_ids[k] = low_synonyms[v[0]]
            else:
                test_verb_ids[k] = v 
                           
        else:
            if None in v:
                v = [i for i in v if i]
            if len(v) == 1:
                if v[0] in high_synonyms:
                    test_verb_ids[k] = high_synonyms[v[0]]
                elif v[0] in low_synonyms:
                    test_verb_ids[k] = low_synonyms[v[0]]
                else:
                    test_verb_ids[k] = v 
            else:
                test_verb_ids[k] = [] 
                #print(k)
                #print(test_verb_ids[k])
                for item in v:
                    
                    if item in high_synonyms:
                        test_verb_ids[k].extend(high_synonyms[item])
                    elif item in low_synonyms:
                        test_verb_ids[k].extend(low_synonyms[item])
                    else:
                        test_verb_ids[k].extend(item) 
    return test_verb_ids 

test_verb_syns = mark_vocab_syns(test_stem_verbs, low_synonyms, synonyms)
train_verb_syns = mark_vocab_syns(train_stem_verbs, low_synonyms, synonyms)

11
[]
12
[]
18
[]
30
[]
32
[]
40
[]
44
[]
54
[]
55
[]
63
[]
64
[]
74
[]
98
[]
106
[]
125
[]
137
[]
138
[]
175
[]
176
[]
203
[]
207
[]
208
[]
220
[]
224
[]
238
[]
239
[]
240
[]
291
[]
296
[]
301
[]
335
[]
348
[]
365
[]
367
[]
371
[]
374
[]
384
[]
395
[]
416
[]
418
[]
420
[]
443
[]
449
[]
458
[]
466
[]
470
[]
473
[]
477
[]
493
[]
509
[]
511
[]
517
[]
518
[]
524
[]
538
[]
550
[]
551
[]
569
[]
587
[]
591
[]
614
[]
629
[]
643
[]
650
[]
661
[]
677
[]
678
[]
682
[]
684
[]
697
[]
711
[]
742
[]
760
[]
763
[]
768
[]
779
[]
804
[]
808
[]
812
[]
837
[]
844
[]
855
[]
859
[]
883
[]
894
[]
922
[]
933
[]
942
[]
948
[]
950
[]
953
[]
984
[]
1019
[]
1024
[]
1066
[]
1084
[]
1094
[]
1098
[]
1129
[]
1133
[]
1147
[]
1154
[]
1199
[]
1206
[]
1214
[]
1220
[]
1225
[]
1232
[]
1236
[]
1253
[]
1261
[]
1278
[]
1290
[]
1295
[]
1297
[]
1299
[]
1302
[]
1327
[]
1330
[]
1331
[]
1336
[]
1342
[]
1361
[]
1368
[]
1388
[]
1395
[]
1423
[]
1426
[]
1431
[]
1437
[]
1448
[]
1450
[]
1455
[]
1479
[]
1481
[]
1485
[]
1487
[]
1497
[]
1

In [81]:
vocab_nouns

{'book': 621,
 'phone': 364,
 'glass': 545,
 'water': 109,
 'television': 139,
 'cup': 311,
 'mirror': 94,
 None: 1731,
 'camera': 143,
 'notebook': 7,
 'door': 1253,
 'shoe': 510,
 'stave': 111,
 'something': 236,
 'coffee': 91,
 'room': 222,
 'couch': 132,
 'bed': 191,
 'laptop': 446,
 'table': 192,
 'sofa': 122,
 'dish': 163,
 'doorknob': 53,
 'light': 786,
 'medicine': 151,
 'chair': 429,
 'refrigerator': 249,
 'clothe': 383,
 'sandwich': 512,
 'blanket': 287,
 'food': 488,
 'blanket/pillow': 1,
 'pillow': 384,
 'bag': 433,
 'floor': 183,
 'box': 400,
 'cabinet': 392,
 'foot': 4,
 'shelf': 98,
 'closet': 221,
 'doorway': 298,
 'bowl': 14,
 'detergent': 5,
 'load': 2,
 'washer': 3,
 'seat': 4,
 'basement': 18,
 'front': 52,
 'window': 424,
 'card': 2,
 'groceries': 71,
 'pan': 3,
 'pair': 55,
 'wardrobe': 16,
 'bathroom': 81,
 'can': 4,
 'bedroom': 65,
 'towel': 251,
 'turn': 4,
 'person': 49,
 'snack': 42,
 'drink': 156,
 'cleaner': 1,
 'stuff': 6,
 'broom': 179,
 'hair': 70,
 'din

In [95]:
from collections import Counter

def count_vocab_train_test(train_verb_syns,test_verb_syns):
    train_verbs = list(train_verb_syns.values())
    test_verbs = list(test_verb_syns.values())
    new_verbs = []
    for item in train_verbs:
        if item:
            for v in item:
                new_verbs.append(v)
    for item in test_verbs:
        if item:
            for v in item:
                new_verbs.append(v)
    c = Counter(new_verbs)
    return c 

def make_vocab(cnt_verb):
    cnt = 0 
    output = {}
    for k,v in cnt_verb.items():
        output[k] = cnt 
        cnt +=1 
    output['null'] = cnt 
    return output 

""" Verb"""
# c = count_vocab_train_test(train_verb_syns,test_verb_syns)
# verb_vocab = make_vocab(c)
""" Nouns"""
c_obj = count_vocab_train_test(train_stem_objs,test_stem_objs)
verb_nouns = make_vocab(c_obj)


def match_ids(test_verb_syns, vocab):
    output_dict= {}
    for k,v in test_verb_syns.items():
        output_dict[k] = [] 
        if v:
            for item in v:
                output_dict[k].append(vocab[item])
        else:
            output_dict[k] = [vocab['null']]
    return output_dict 

def match_ids_nouns(test_verb_syns, vocab):
    output_dict= {}
    for k,v in test_verb_syns.items():
        output_dict[k] = [] 
        print("keys", k )
        if v:
            if 'int' in str(type(v)): 
                output_dict[k].append(vocab[v])
            else:
                for item in v:
                    output_dict[k].append(vocab[item])
        else:
            output_dict[k] = [vocab[None]]
    return output_dict 

test_verb_ids = match_ids(test_verb_syns, verb_vocab)
train_verb_ids = match_ids(train_verb_syns, verb_vocab)

#test_noun_ids = match_ids_nouns(test_stem_verbs, vocab_nouns)
#train_noun_ids = match_ids_nouns(train_stem_verbs, vocab_nouns)

def convert_ids(test_verb_ids, vocab):
    output_dict = {}
    for k,v in test_verb_ids.items():
        output_dict[k] = list([0]*(len(vocab)))
        for _id in v:
            output_dict[k][_id] = 1 
    return output_dict

test_verb_ids_one = convert_ids(test_verb_ids, verb_vocab)
train_verb_ids_one = convert_ids(train_verb_ids, verb_vocab)

keys 0


KeyError: 23

In [102]:
test_noun_ids = match_ids_nouns(test_stem_objs, verb_nouns)
train_noun_ids = match_ids_nouns(train_stem_objs, verb_nouns)

keys 0
keys 1
keys 2
keys 3
keys 4
keys 5
keys 6
keys 7
keys 8
keys 9
keys 10
keys 11
keys 12
keys 13
keys 14
keys 15
keys 16
keys 17
keys 18
keys 19
keys 20
keys 21
keys 22
keys 23
keys 24
keys 25
keys 26
keys 27
keys 28
keys 29
keys 30
keys 31
keys 32
keys 33
keys 34
keys 35
keys 36
keys 37
keys 38
keys 39
keys 40
keys 41
keys 42
keys 43
keys 44
keys 45
keys 46
keys 47
keys 48
keys 49
keys 50
keys 51
keys 52
keys 53
keys 54
keys 55
keys 56
keys 57
keys 58
keys 59
keys 60
keys 61
keys 62
keys 63
keys 64
keys 65
keys 66
keys 67
keys 68
keys 69
keys 70
keys 71
keys 72
keys 73
keys 74
keys 75
keys 76
keys 77
keys 78
keys 79
keys 80
keys 81
keys 82
keys 83
keys 84
keys 85
keys 86
keys 87
keys 88
keys 89
keys 90
keys 91
keys 92
keys 93
keys 94
keys 95
keys 96
keys 97
keys 98
keys 99
keys 100
keys 101
keys 102
keys 103
keys 104
keys 105
keys 106
keys 107
keys 108
keys 109
keys 110
keys 111
keys 112
keys 113
keys 114
keys 115
keys 116
keys 117
keys 118
keys 119
keys 120
keys 121
keys 122
key

keys 1502
keys 1503
keys 1504
keys 1505
keys 1506
keys 1507
keys 1508
keys 1509
keys 1510
keys 1511
keys 1512
keys 1513
keys 1514
keys 1515
keys 1516
keys 1517
keys 1518
keys 1519
keys 1520
keys 1521
keys 1522
keys 1523
keys 1524
keys 1525
keys 1526
keys 1527
keys 1528
keys 1529
keys 1530
keys 1531
keys 1532
keys 1533
keys 1534
keys 1535
keys 1536
keys 1537
keys 1538
keys 1539
keys 1540
keys 1541
keys 1542
keys 1543
keys 1544
keys 1545
keys 1546
keys 1547
keys 1548
keys 1549
keys 1550
keys 1551
keys 1552
keys 1553
keys 1554
keys 1555
keys 1556
keys 1557
keys 1558
keys 1559
keys 1560
keys 1561
keys 1562
keys 1563
keys 1564
keys 1565
keys 1566
keys 1567
keys 1568
keys 1569
keys 1570
keys 1571
keys 1572
keys 1573
keys 1574
keys 1575
keys 1576
keys 1577
keys 1578
keys 1579
keys 1580
keys 1581
keys 1582
keys 1583
keys 1584
keys 1585
keys 1586
keys 1587
keys 1588
keys 1589
keys 1590
keys 1591
keys 1592
keys 1593
keys 1594
keys 1595
keys 1596
keys 1597
keys 1598
keys 1599
keys 1600
keys 1601


keys 2349
keys 2350
keys 2351
keys 2352
keys 2353
keys 2354
keys 2355
keys 2356
keys 2357
keys 2358
keys 2359
keys 2360
keys 2361
keys 2362
keys 2363
keys 2364
keys 2365
keys 2366
keys 2367
keys 2368
keys 2369
keys 2370
keys 2371
keys 2372
keys 2373
keys 2374
keys 2375
keys 2376
keys 2377
keys 2378
keys 2379
keys 2380
keys 2381
keys 2382
keys 2383
keys 2384
keys 2385
keys 2386
keys 2387
keys 2388
keys 2389
keys 2390
keys 2391
keys 2392
keys 2393
keys 2394
keys 2395
keys 2396
keys 2397
keys 2398
keys 2399
keys 2400
keys 2401
keys 2402
keys 2403
keys 2404
keys 2405
keys 2406
keys 2407
keys 2408
keys 2409
keys 2410
keys 2411
keys 2412
keys 2413
keys 2414
keys 2415
keys 2416
keys 2417
keys 2418
keys 2419
keys 2420
keys 2421
keys 2422
keys 2423
keys 2424
keys 2425
keys 2426
keys 2427
keys 2428
keys 2429
keys 2430
keys 2431
keys 2432
keys 2433
keys 2434
keys 2435
keys 2436
keys 2437
keys 2438
keys 2439
keys 2440
keys 2441
keys 2442
keys 2443
keys 2444
keys 2445
keys 2446
keys 2447
keys 2448


keys 3252
keys 3253
keys 3254
keys 3255
keys 3256
keys 3257
keys 3258
keys 3259
keys 3260
keys 3261
keys 3262
keys 3263
keys 3264
keys 3265
keys 3266
keys 3267
keys 3268
keys 3269
keys 3270
keys 3271
keys 3272
keys 3273
keys 3274
keys 3275
keys 3276
keys 3277
keys 3278
keys 3279
keys 3280
keys 3281
keys 3282
keys 3283
keys 3284
keys 3285
keys 3286
keys 3287
keys 3288
keys 3289
keys 3290
keys 3291
keys 3292
keys 3293
keys 3294
keys 3295
keys 3296
keys 3297
keys 3298
keys 3299
keys 3300
keys 3301
keys 3302
keys 3303
keys 3304
keys 3305
keys 3306
keys 3307
keys 3308
keys 3309
keys 3310
keys 3311
keys 3312
keys 3313
keys 3314
keys 3315
keys 3316
keys 3317
keys 3318
keys 3319
keys 3320
keys 3321
keys 3322
keys 3323
keys 3324
keys 3325
keys 3326
keys 3327
keys 3328
keys 3329
keys 3330
keys 3331
keys 3332
keys 3333
keys 3334
keys 3335
keys 3336
keys 3337
keys 3338
keys 3339
keys 3340
keys 3341
keys 3342
keys 3343
keys 3344
keys 3345
keys 3346
keys 3347
keys 3348
keys 3349
keys 3350
keys 3351


keys 1150
keys 1151
keys 1152
keys 1153
keys 1154
keys 1155
keys 1156
keys 1157
keys 1158
keys 1159
keys 1160
keys 1161
keys 1162
keys 1163
keys 1164
keys 1165
keys 1166
keys 1167
keys 1168
keys 1169
keys 1170
keys 1171
keys 1172
keys 1173
keys 1174
keys 1175
keys 1176
keys 1177
keys 1178
keys 1179
keys 1180
keys 1181
keys 1182
keys 1183
keys 1184
keys 1185
keys 1186
keys 1187
keys 1188
keys 1189
keys 1190
keys 1191
keys 1192
keys 1193
keys 1194
keys 1195
keys 1196
keys 1197
keys 1198
keys 1199
keys 1200
keys 1201
keys 1202
keys 1203
keys 1204
keys 1205
keys 1206
keys 1207
keys 1208
keys 1209
keys 1210
keys 1211
keys 1212
keys 1213
keys 1214
keys 1215
keys 1216
keys 1217
keys 1218
keys 1219
keys 1220
keys 1221
keys 1222
keys 1223
keys 1224
keys 1225
keys 1226
keys 1227
keys 1228
keys 1229
keys 1230
keys 1231
keys 1232
keys 1233
keys 1234
keys 1235
keys 1236
keys 1237
keys 1238
keys 1239
keys 1240
keys 1241
keys 1242
keys 1243
keys 1244
keys 1245
keys 1246
keys 1247
keys 1248
keys 1249


...

keys 7779
keys 7780
keys 7781
keys 7782
keys 7783
keys 7784
keys 7785
keys 7786
keys 7787
keys 7788
keys 7789
keys 7790
keys 7791
keys 7792
keys 7793
keys 7794
keys 7795
keys 7796
keys 7797
keys 7798
keys 7799
keys 7800
keys 7801
keys 7802
keys 7803
keys 7804
keys 7805
keys 7806
keys 7807
keys 7808
keys 7809
keys 7810
keys 7811
keys 7812
keys 7813
keys 7814
keys 7815
keys 7816
keys 7817
keys 7818
keys 7819
keys 7820
keys 7821
keys 7822
keys 7823
keys 7824
keys 7825
keys 7826
keys 7827
keys 7828
keys 7829
keys 7830
keys 7831
keys 7832
keys 7833
keys 7834
keys 7835
keys 7836
keys 7837
keys 7838
keys 7839
keys 7840
keys 7841
keys 7842
keys 7843
keys 7844
keys 7845
keys 7846
keys 7847
keys 7848
keys 7849
keys 7850
keys 7851
keys 7852
keys 7853
keys 7854
keys 7855
keys 7856
keys 7857
keys 7858
keys 7859
keys 7860
keys 7861
keys 7862
keys 7863
keys 7864
keys 7865
keys 7866
keys 7867
keys 7868
keys 7869
keys 7870
keys 7871
keys 7872
keys 7873
keys 7874
keys 7875
keys 7876
keys 7877
keys 7878


keys 8779
keys 8780
keys 8781
keys 8782
keys 8783
keys 8784
keys 8785
keys 8786
keys 8787
keys 8788
keys 8789
keys 8790
keys 8791
keys 8792
keys 8793
keys 8794
keys 8795
keys 8796
keys 8797
keys 8798
keys 8799
keys 8800
keys 8801
keys 8802
keys 8803
keys 8804
keys 8805
keys 8806
keys 8807
keys 8808
keys 8809
keys 8810
keys 8811
keys 8812
keys 8813
keys 8814
keys 8815
keys 8816
keys 8817
keys 8818
keys 8819
keys 8820
keys 8821
keys 8822
keys 8823
keys 8824
keys 8825
keys 8826
keys 8827
keys 8828
keys 8829
keys 8830
keys 8831
keys 8832
keys 8833
keys 8834
keys 8835
keys 8836
keys 8837
keys 8838
keys 8839
keys 8840
keys 8841
keys 8842
keys 8843
keys 8844
keys 8845
keys 8846
keys 8847
keys 8848
keys 8849
keys 8850
keys 8851
keys 8852
keys 8853
keys 8854
keys 8855
keys 8856
keys 8857
keys 8858
keys 8859
keys 8860
keys 8861
keys 8862
keys 8863
keys 8864
keys 8865
keys 8866
keys 8867
keys 8868
keys 8869
keys 8870
keys 8871
keys 8872
keys 8873
keys 8874
keys 8875
keys 8876
keys 8877
keys 8878


keys 9779
keys 9780
keys 9781
keys 9782
keys 9783
keys 9784
keys 9785
keys 9786
keys 9787
keys 9788
keys 9789
keys 9790
keys 9791
keys 9792
keys 9793
keys 9794
keys 9795
keys 9796
keys 9797
keys 9798
keys 9799
keys 9800
keys 9801
keys 9802
keys 9803
keys 9804
keys 9805
keys 9806
keys 9807
keys 9808
keys 9809
keys 9810
keys 9811
keys 9812
keys 9813
keys 9814
keys 9815
keys 9816
keys 9817
keys 9818
keys 9819
keys 9820
keys 9821
keys 9822
keys 9823
keys 9824
keys 9825
keys 9826
keys 9827
keys 9828
keys 9829
keys 9830
keys 9831
keys 9832
keys 9833
keys 9834
keys 9835
keys 9836
keys 9837
keys 9838
keys 9839
keys 9840
keys 9841
keys 9842
keys 9843
keys 9844
keys 9845
keys 9846
keys 9847
keys 9848
keys 9849
keys 9850
keys 9851
keys 9852
keys 9853
keys 9854
keys 9855
keys 9856
keys 9857
keys 9858
keys 9859
keys 9860
keys 9861
keys 9862
keys 9863
keys 9864
keys 9865
keys 9866
keys 9867
keys 9868
keys 9869
keys 9870
keys 9871
keys 9872
keys 9873
keys 9874
keys 9875
keys 9876
keys 9877
keys 9878


keys 10779
keys 10780
keys 10781
keys 10782
keys 10783
keys 10784
keys 10785
keys 10786
keys 10787
keys 10788
keys 10789
keys 10790
keys 10791
keys 10792
keys 10793
keys 10794
keys 10795
keys 10796
keys 10797
keys 10798
keys 10799
keys 10800
keys 10801
keys 10802
keys 10803
keys 10804
keys 10805
keys 10806
keys 10807
keys 10808
keys 10809
keys 10810
keys 10811
keys 10812
keys 10813
keys 10814
keys 10815
keys 10816
keys 10817
keys 10818
keys 10819
keys 10820
keys 10821
keys 10822
keys 10823
keys 10824
keys 10825
keys 10826
keys 10827
keys 10828
keys 10829
keys 10830
keys 10831
keys 10832
keys 10833
keys 10834
keys 10835
keys 10836
keys 10837
keys 10838
keys 10839
keys 10840
keys 10841
keys 10842
keys 10843
keys 10844
keys 10845
keys 10846
keys 10847
keys 10848
keys 10849
keys 10850
keys 10851
keys 10852
keys 10853
keys 10854
keys 10855
keys 10856
keys 10857
keys 10858
keys 10859
keys 10860
keys 10861
keys 10862
keys 10863
keys 10864
keys 10865
keys 10866
keys 10867
keys 10868
keys 10869

keys 11778
keys 11779
keys 11780
keys 11781
keys 11782
keys 11783
keys 11784
keys 11785
keys 11786
keys 11787
keys 11788
keys 11789
keys 11790
keys 11791
keys 11792
keys 11793
keys 11794
keys 11795
keys 11796
keys 11797
keys 11798
keys 11799
keys 11800
keys 11801
keys 11802
keys 11803
keys 11804
keys 11805
keys 11806
keys 11807
keys 11808
keys 11809
keys 11810
keys 11811
keys 11812
keys 11813
keys 11814
keys 11815
keys 11816
keys 11817
keys 11818
keys 11819
keys 11820
keys 11821
keys 11822
keys 11823
keys 11824
keys 11825
keys 11826
keys 11827
keys 11828
keys 11829
keys 11830
keys 11831
keys 11832
keys 11833
keys 11834
keys 11835
keys 11836
keys 11837
keys 11838
keys 11839
keys 11840
keys 11841
keys 11842
keys 11843
keys 11844
keys 11845
keys 11846
keys 11847
keys 11848
keys 11849
keys 11850
keys 11851
keys 11852
keys 11853
keys 11854
keys 11855
keys 11856
keys 11857
keys 11858
keys 11859
keys 11860
keys 11861
keys 11862
keys 11863
keys 11864
keys 11865
keys 11866
keys 11867
keys 11868

In [104]:
verb_nouns
vocab_idx_val = {v:k for k,v in verb_nouns.items()}

In [106]:
with open('train_obj_idx.json', 'w') as f:
    json.dump([train_noun_ids, train_stem_objs], f)
    
with open('test_obj_idx.json', 'w') as f:
    json.dump([test_noun_ids, test_stem_objs], f)
    
with open('vocab_obj.json', 'w') as f:
    json.dump([verb_nouns, vocab_idx_val], f)
    


In [77]:
import json 
with open('train_verb_idx_syn.json', 'w') as f:
    json.dump([train_verb_ids, train_verb_ids_one, train_verb_syns], f)

with open('test_verb_idx_syn.json', 'w') as f:
    json.dump([test_verb_ids, test_verb_ids_one, test_verb_syns], f)
    
with open('verb_vocab.json', 'w') as f:
    json.dump([verb_vocab], f)
    
with open('nouns_vocab.json', 'w') as f:
    json.dump([vocab_nouns], f)
    

In [108]:
vocab_verb_id = {v:k for k,v in verb_vocab.items()}
with open('verb_vocab.json', 'w') as f:
    json.dump([verb_vocab, vocab_verb_id], f)


In [49]:
train_verb_ids_one

{0: array([1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]),
 1: array([0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        

In [ ]:
obj_v_ind
oj_words = {v:k for k,v in obj_v_ind.items()}
with open('obj_vocab.json', 'w') as f:
    json.dump([oj_words, obj_v_ind], f) 

In [ ]:
import json 

with open('train_obj_idx.json', 'w') as f:
    json.dump([train_obj_ids, train_stem_objs], f)
    
with open('test_obj_idx.json', 'w') as f:
    json.dump([test_obj_ids, test_stem_objs], f)
    

In [ ]:
train_verb_ids = {}
for k,v in train_stem_verbs.items():
    if len(v) == 0:
        train_verb_ids[k] = action_v_ind[None] 
    elif len(v) == 1:
        train_verb_ids[k] = action_v_ind[v[0]]
    else:
        if None in v:
            v.remove(None)
        print("clean v", v)
        if len(v) == 1:
            train_verb_ids[k] = action_v_ind[v[0]]
        else:
            v = list(filter(lambda a: a != None, v))
            item="_".join(v)
            train_verb_ids[k] = action_v_ind[item]

In [ ]:
# Queries verb and the tokens 
train_verb_tks = {}
for k,v in train_verb_ids.items():
    train_verb_tks[k] = action_index[v] 

test_verb_tks = {}
for k,v in test_verb_ids.items():
    test_verb_tks[k] = action_index[v] 
    

In [ ]:
import json 
with open('train_verb_idx.json', 'w') as f:
    json.dump([train_verb_ids, train_verb_tks], f)

In [ ]:
with open('test_verb_idx.json', 'w') as f:
    json.dump([test_verb_ids, test_verb_tks], f)

In [ ]:
with open('verb_vocab.json', 'w') as f:
    json.dump([action_index, action_v_ind], f) 

In [ ]:
test_nouns

In [ ]:
import csv
with open("test_verb_tks.csv","w") as cf:
    wr = csv.writer(cf)
    for k,v in test_verb_tks.items():
        wr.writerow([k,v])


In [ ]:
with open("test_nouns_tks.csv","w") as cf:
    wr = csv.writer(cf)
    for k,v in test_stem_objs.items():
        wr.writerow([k,v])

In [ ]:
import pandas as pd 
else_data = pd.read_csv("test_bad_verbs.csv",header=None)
from collections import Counter
vbs = Counter(list(else_data[1]))
vbs

In [ ]:
else_nouns = pd.read_csv("test_bad_nouns.csv",header=None)
objs = Counter(list(else_nouns[1]))
objs